In [5]:
import os
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from openai import OpenAI
from prompts import question_prompt, assessment_prompt
# Load environment variables
load_dotenv()


# Access email and password from .env file
email_id = os.getenv("EMAIL")
password = os.getenv("PASSWORD")

def login():
    # Wait until the email input field is present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div/section/section/main/div/div[1]/main/form[1]/div[1]/div[1]/input'))
    )

    # Find the email input field and enter the email
    email_input = driver.find_element(By.XPATH, '/html/body/div/section/section/main/div/div[1]/main/form[1]/div[1]/div[1]/input')
    email_input.send_keys(email_id)

    # Find the password input field and enter the password
    password_input = driver.find_element(By.XPATH, '/html/body/div/section/section/main/div/div[1]/main/form[1]/div[2]/div[1]/input')
    password_input.send_keys(password)

    # Submit the form
    password_input.send_keys(Keys.RETURN)
    

def click_optional_buttons():
    # optional button 1
    # Click the specified button
    try:
        button = driver.find_element(By.XPATH, '/html/body/div/section/section/main/div/main/div[1]/div/div[1]/div[1]/div/div[3]/button')
        button.click()
    except:
        pass
    try:
        # optional button 2
        # Click the got it button
        button = driver.find_element(By.XPATH, '/html/body/div/div[1]/section[1]/div[2]/div/button')
        button.click()
    except:
        pass
    try:
        # optional button 3
        # Click the got it button
        button = driver.find_element(By.XPATH, '/html/body/div/div[1]/section[2]/div[1]/div/button')
        button.click()
    except:
        pass
    try:
        # optional button 4
        # Click the "Take the Level 2 Assessment" Button
        button = driver.find_element(By.XPATH, '/html/body/div/section/section/main/div/main/nav/div/button')
        button.click()
    except:
        pass
    try:
        # optional button 5
        # Click the "Submit Assessment" Button
        button = driver.find_element(By.XPATH, '/html/body/div/div[1]/div/section/div[2]/footer/button[1]')
        button.click()
    except:
        pass
    try:
        # optional button 6
        # Click the "Continue" Button after completing the assessment
        button = driver.find_element(By.XPATH, '/html/body/div/section/section/main/div/div/div[1]/button')
        button.click()
    except:
        pass
    try:
        # optional button 7
        # Click the "No thanks" Button for ads
        button = driver.find_element(By.XPATH, '/html/body/div/div[1]/div/section/div[2]/main/div/div[2]/button[2]')
        button.click()
    except:
        pass
    try:
        # optional button 8
        # Click the Video Play button
        button = driver.find_element(By.XPATH, '/html/body/div/section/section/main/main/nav/div/div[3]/button')
        button.click()
    except:
        pass
    try:
        # optional button 9
        # Click the Video play button
        button = driver.find_element(By.XPATH, '/html/body/div/section/section/main/main/article/div/div[1]/div/button')
        button.click()
    except:
        pass
    
    

def check_question_section():
    # Check if the specific section exists
    try:
        section = driver.find_element(By.XPATH, '/html/body/div/section/section/main/main/article/div')
        if section.get_attribute("data-test") == "popupQuestionContainer":
            print(True)
            print("The Question section exists on the page.")
            question_section_html = section.get_attribute('outerHTML')
            return question_section_html
        else:
            pass
    except:
        pass
    return None

def check_assessment_section():
    try:
        section = driver.find_element(By.XPATH, '/html/body/div/section/section/main/div/article/div')
        if section.get_attribute("class") == "gritCourseflowNode__question-wrapper":
            print(True)
            print("The Question section exists on the page.")
            assessment_section_html = section.get_attribute('outerHTML')
            return assessment_section_html
        else:
            pass
    except:
        pass
    return None

def answer_assessment(answer_text):
    try:
        escaped_text = answer_text.replace("'", "\'")
    
        # Use an XPath with normalize-space() and contains() to handle multiline and special characters
        button = driver.find_element(By.XPATH, f"//button[contains(normalize-space(), \"{escaped_text}\")]")
        button.click()
        print(f"Clicked the button with text {answer_text}.")
        try:
            submit_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/div/section/section/main/div/nav/div/div[3]/button'))
            )
            submit_button.click()
            print("Clicked on the submit button.")
        except Exception as e:
            print(f"Error occured while clicking Submit. {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

def answer_question(answer_button_Xpath):
    try:
        answer_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, answer_button_Xpath))
        )
        answer_button.click()
        print("Clicked on the answer button.")
        try:
            submit_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/div/section/section/main/main/nav/div/div[3]/button'))
            )
            submit_button.click()
            print("Clicked on the submit button.")
            answer_status = answer_status = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH,'/html/body/div/section/section/main/main/article/div/div[2]/div/div[1]/div[2]/div/div/h3'))
                ).text
            return answer_status
        except Exception as e:
            print(f"An error occurred while clicking the submit button: {e}")
    except Exception as e:
        print(f"An error occurred while clicking the answer button: {e}")
    return None

In [6]:
from openai import OpenAI

def get_gpt4o_response(prompt, model="gpt-4o", temperature=0, max_tokens=300):
    """
    Function to send a prompt to the OpenAI GPT-4 model and retrieve a response.

    Parameters:
        prompt (str): The input prompt to send to the GPT model.
        model (str): The model to use (default is "gpt-4").
        temperature (float): Sampling temperature. Higher values make the output more random.
        max_tokens (int): The maximum number of tokens to include in the response.

    Returns:
        str: The model's response to the input prompt.
    """
    try:
        
        client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY")
        )
        # Send the prompt to the OpenAI API
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt,}],
            temperature=temperature,
            model=model,
        )

        # Extract and return the response content
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {str(e)}"


In [7]:
# Initialize WebDriver (use ChromeDriver as an example)
service = Service()  # Update with the path to your ChromeDriver
options = webdriver.ChromeOptions()

# Optional: Add any additional options (e.g., headless mode)
# options.add_argument('--headless')

# Start the WebDriver
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://app.aceable.com/user/signin")
# login to the account
login()
# Wait for the page to load after login
page = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, 'body'))
)


In [ ]:
click_optional_buttons()
click_optional_buttons()

In [ ]:
while True:
    click_optional_buttons()

    # Answer Questions during Course
    question_section_html = check_question_section()
    if question_section_html!=None:
        prompt = question_prompt.format(question_section_html=question_section_html)
        print(f"Pinging gpt4o...\n\n{prompt}\n\n")
        answer_button_Xpath = get_gpt4o_response(prompt).strip().split("'")[-2]
        print(f"Response:{answer_button_Xpath}\n\n")
        answer_status = answer_question(answer_button_Xpath)
#         if "Correct" in answer_status:
#             print("Answer is Correct")
#         else:
#             print("Answer is Wrong")
#             break

    # Answer Assessment Questions
    assessment_section_html = check_assessment_section()
    if assessment_section_html!=None:
        prompt = assessment_prompt.format(assessment_section_html=assessment_section_html)
        print(f"Pinging gpt4o...\n\n{prompt}\n\n")
        response = get_gpt4o_response(prompt).strip()
        if "python" in response:
            answer_text = response.split('"')[-2]
        else:
            answer_text = response.strip().split("'")[-2]
        print(f"Response: {answer_text}\n\n")
        answer_status = answer_assessment(answer_text)

    # Wait for the timer countdown to finish and click the button
    try:
        next_button = driver.find_element(By.XPATH, '/html/body/div/section/section/main/main/nav/div/div[3]/button')
        try:
            timer_element = driver.find_element(By.XPATH, '/html/body/div/section/section/main/main/nav/div/div[3]/button/span[1]/div')
            timer_text = timer_element.text
            
            # Parse the timer text into seconds
            if ":" in timer_text:
                minutes, seconds = map(int, timer_text.split(":"))
                wait_time = minutes * 60 + seconds
            else:
                wait_time = int(timer_text)

            # Wait for the specified time
            print(f"Waiting for {wait_time} seconds for the timer to reach 00:00.")
            time.sleep(wait_time)
            next_button.click()
        except:
            print("Timer not found, just clicking next button")
            # Click the button

            next_button.click()

    except Exception as e:
        print(f"An error occurred while waiting for the timer or clicking the next button: {e}")
    time.sleep(3)
